In [3]:
# Model Training and Evaluation

# Retrieve previously stored variables
%store -r x_train # train samples: 2185
%store -r x_test # testing samples: 547
%store -r y_train # categories train: 2185
%store -r y_test # 
%store -r yy # total samples
%store -r le

no stored variable # train samples: 2185
no stored variable # testing samples: 547
no stored variable # categories train: 2185
no stored variable #
no stored variable # total samples


In [8]:
display(x_train[1])
print((y_train))
display((yy.shape[1]))
print(len(x_test))
print(x_train.shape)

array([-3.19514862e+02,  1.39700089e+02, -1.12087608e+02, -4.92915726e+01,
       -1.74340093e+00, -5.69976540e+01, -4.37130013e+01, -6.43806553e+00,
       -1.52856960e+01, -2.53208313e+01, -1.17009325e+01, -7.51425409e+00,
       -9.69702148e+00, -2.19402933e+00,  7.07967758e+00,  1.73335800e+01,
        1.20680609e+01, -1.13923674e+01, -1.51345863e+01, -3.63308001e+00,
        3.87587762e+00,  1.19124365e+01,  5.14146280e+00, -9.87630939e+00,
       -6.61446285e+00,  2.29413822e-01, -5.05282927e+00, -3.69123459e+00,
        7.10211515e+00,  8.15948486e+00,  2.64397812e+00,  9.79319692e-01,
       -1.59480238e+00, -1.82388520e+00,  2.49361086e+00,  5.21429539e+00,
        1.82872808e+00, -6.47891617e+00, -5.83984756e+00,  2.10689330e+00],
      dtype=float32)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


4

547
(2185, 40)


In [10]:
# Import libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

# Extract the number of labels - 4 in our case
num_labels = yy.shape[1]

# Begin constructing ML model
# Create an object of the Sequential class
model = Sequential()

# Create input layer using the Dense function
numNodes = 64; # Trial and error - 1/4 of 256 bit
numMFCC = 40;
# Input shape is the size of the input array (1-D array of 40 columns, 1 row)
model.add(Dense(numNodes, input_shape=(numMFCC,)));
# Specifying the activation function to be used - relu: Rectified Linear Activiation
model.add(Activation('relu'))
# Dropout value of 50% - means random half of neurons exluded from each update cycle. Used to prevent overfitting.
model.add(Dropout(0.5))


# Add subsequest hidden layer - DENSE function
model.add(Dense(numNodes))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Add the output layer - DENSE function
# Output nodes is the different categories
# Different usage of activation function
model.add(Dense(num_labels))
model.add(Activation('softmax'))

Using TensorFlow backend.


In [11]:
# Compile the model
# Optimizer "adam" is a typical optimizer used - variation SGD (stochastic gradient descent)
# SGD utilizes the gradient of the loss function with respects to the weight
# loss -> typical loss function 
# metrics is output to be displayed (accuracy is the output of the loss function (?))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [15]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
# Verbose - displays info if desired (verbose = 0 means silent, just print accuracy value)
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

# Accuracy - the metrics value evaluated based on loss function
print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                2624      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                

In [17]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 8 # Arbitrarily chose the value 8

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 2185 samples, validate on 547 samples
Epoch 1/100
2185/2185 [==============================] - 1s 305us/step - loss: 3.8993 - accuracy: 0.4142 - val_loss: 0.9565 - val_accuracy: 0.5375

Epoch 00001: val_loss improved from inf to 0.95651, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
2185/2185 [==============================] - 1s 327us/step - loss: 1.9900 - accuracy: 0.4462 - val_loss: 0.9463 - val_accuracy: 0.5923

Epoch 00002: val_loss improved from 0.95651 to 0.94633, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
2185/2185 [==============================] - 1s 309us/step - loss: 1.3276 - accuracy: 0.5140 - val_loss: 0.8587 - val_accuracy: 0.7020

Epoch 00003: val_loss improved from 0.94633 to 0.85869, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
2185/2185 [==============================] - 1s 304us/step - loss: 1.1462 - accuracy: 0.5579 - val_loss: 0.8251 - val_accuracy: 0.7239

Epoch 00004: val_loss impr

In [19]:
# Test the Model

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9102974534034729
Testing Accuracy:  0.9012796878814697


In [20]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])

In [21]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [24]:
import os

# Random dog bark file
cwd = os.getcwd()
filename = cwd + '//UrbanSound8K//audio//fold1//101415-3-0-2.wav'
print_prediction(filename)

The predicted class is: dog_bark 

car_horn 		 :  0.00000000000000017026275447207695
dog_bark 		 :  1.00000000000000000000000000000000
gun_shot 		 :  0.00000000000020038213709855368094
siren 		 :  0.00000000000519648863869792698722


In [26]:
# Air conditioner file
cwd = os.getcwd()
filename = cwd + '//UrbanSound8K//audio//fold5//100852-0-0-0.wav'
print_prediction(filename)

The predicted class is: car_horn 

car_horn 		 :  0.96730029582977294921875000000000
dog_bark 		 :  0.03246358036994934082031250000000
gun_shot 		 :  0.00023617666738573461771011352539
siren 		 :  0.00000002034192014832569839200005
